In [1]:
#  这是写好了的，勿动

import os, pickle, random
import numpy as np
import torch

In [2]:
# change the current directory to the father directory of this file
os.chdir(os.path.dirname(os.path.dirname(os.path.realpath('evaluate.ipynb'))))
print(os.getcwd())

/home/yangshuaiyu6791/RepresentAttack/inversion_sePoint4


In [3]:
from modules.models import sPoints_net, ePoints_net, se_net
from DataLoader.dataloader import TrajDataset
emb_dim = 256
emb_model = 'start'

# load data
with open("./data/{}/{}/train/train_data".format(emb_model, emb_dim), "rb") as f:
    tmp_embs = pickle.load(f)
    train_embs = tmp_embs['embs']
with open("./data/{}/{}/train/train_norm".format(emb_model, emb_dim), "rb") as f:
    train_norms = pickle.load(f)

with open("./data/{}/{}/val/val_data".format(emb_model, emb_dim), "rb") as f:
    tmp_embs = pickle.load(f)
    val_embs = tmp_embs['embs']
with open("./data/{}/{}/val/val_norm".format(emb_model, emb_dim), "rb") as f:
    val_norms = pickle.load(f)

with open("./data/{}/{}/test/test_data".format(emb_model, emb_dim), "rb") as f:
    tmp_embs = pickle.load(f)
    test_embs = tmp_embs['embs']
with open("./data/{}/{}/test/test_norm".format(emb_model, emb_dim), "rb") as f:
    test_norms = pickle.load(f)
    
with open("./rdata/gps_seqs/traingps", "rb") as f:
    train_gps = pickle.load(f)
    train_gps = train_gps[0:60000]
with open("./rdata/gps_seqs/valgps", "rb") as f:
    val_gps = pickle.load(f)
with open("./rdata/gps_seqs/testgps", "rb") as f:
    test_gps = pickle.load(f)

In [4]:
# load pth
# delta = 0.002, emb_model = neutraj, embedding dim = emb_dim, hidden dim = 512

model = se_net(emb_dim, 512)
params = torch.load('./models/d0_001_train_sePoints_{}_{}_{}.pth'.format(emb_model, emb_dim, 512))
model.load_state_dict(params)

<All keys matched successfully>

In [5]:
# calculate the distance between two points with longitude and latitude
def distance(lon_lat_list):
    lon1, lat1, lon2, lat2 = lon_lat_list
    R = 6371.004    # km
    pi = 3.1415926
    dlat = (lat2 - lat1) * pi / 180
    dlon = (lon2 - lon1) * pi / 180
    a = np.sin(dlat / 2) * np.sin(dlat / 2) + np.cos(lat1 * pi / 180) * np.cos(lat2 * pi / 180) * np.sin(dlon / 2) * np.sin(dlon / 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = R * c
    return d

# transform the cell x,y to lon, lat
porto_lat_range = [40.953673,41.307945]
porto_lon_range = [-8.735152,-8.156309]

def pred2gps(pred):
    lon = pred[0] * (porto_lon_range[1] - porto_lon_range[0]) + porto_lon_range[0]
    lat = pred[1] * (porto_lat_range[1] - porto_lat_range[0]) + porto_lat_range[0]
    return [lon, lat]

In [7]:
test_embs_ = np.array(test_embs)    # shape: (test_size, dim)
test_norms_ = np.array([[tnorm[0], tnorm[-1]] for tnorm in test_norms])    # shape: (test_size, 2, 2)
test_gps_ = np.array([[gps[0], gps[-1]] for gps in test_gps])    # shape: (test_size, 2,2)

model.cpu(); model.eval()
test_preds = model(torch.from_numpy(test_embs_).float())
test_preds = test_preds.detach().numpy().reshape(-1, 2, 2)    # shape: (test_size, 4) -- s_lon, s_lat, e_lon, e_lat 

# get the first and last point of every trajectory in test_norms
mean = np.array([(porto_lon_range[1] + porto_lon_range[0])/2, (porto_lat_range[1] + porto_lat_range[0])/2])
std = np.array([(porto_lon_range[1] - porto_lon_range[0])/2, (porto_lat_range[1] - porto_lat_range[0])/2])

test_predgps_ = test_preds * std + mean
test_gps_ = test_gps_ 

test_dist = np.concatenate((test_predgps_, test_gps_), axis=2)    # shape: (test_size, 2, 4) -- pred_lon, pred_lat, gps_lon, gps_lat
print(test_dist.shape)

# calculate dist
dist = np.apply_along_axis(distance, 2, test_dist)
print('dist: ', dist.shape)
mean_dis = dist.mean(axis=0)
print('mean_start_dis: ', mean_dis[0])
print('mean_end_dis: ', mean_dis[1])
print('mean_dis: ', mean_dis.mean())

with open('./results/{}_{}_{}_dists'.format(emb_model, emb_dim, 512), 'w') as f:
    f.write('mean_start_dis: {}\n'.format(mean_dis[0]))
    f.write('mean_end_dis: {}\n'.format(mean_dis[1]))
    f.write('mean_dis: {}\n'.format(mean_dis.mean()))
    f.close()


(10000, 2, 4)
dist:  (10000, 2)
mean_start_dis:  0.5496174912742913
mean_end_dis:  0.6045162932923478
mean_dis:  0.5770668922833195


In [8]:
''' not used orz'''
# # choose 40 points by random
# choice_list = np.random.choice(len(val_embs), 10000, replace=False)
# print([val_gps[i][0] for i in choice_list])    # lon, lat
# choice_testnorms = [[test_norms[i][0][0], val_norms[i][0][1], test_norms[i][-1][0], test_norms[i][-1][1]] for i in choice_list]    # x, y
# choice_testembs = [test_embs[i] for i in choice_list]
# choice_testgps = [[test_gps[i][0], test_gps[i][-1]] for i in choice_list]    # [[lon, lat], [lon, lat]]
# # get the prediction
# model.cpu()
# model.eval()
# choice_predgrids = []
# for i in range(len(choice_testembs)):    
#     choice_testembs[i] = torch.Tensor(choice_testembs[i]).unsqueeze(0)
#     choice_predgrids.append(model(choice_testembs[i]).cpu().detach().numpy()[0])

# # for i in range(len(choice_predgrids)):
# #     print(choice_predgrids[i], choice_valgrids[i], '\n')

# # transform the prediction to lon, lat
# choice_predgps = []
# for i in range(len(choice_predgrids)):
#     # choice_predgps.append([choice_predgrids[i][0:2], choice_predgrids[i][2:]])
#     choice_predgps.append([pred2gps(choice_predgrids[i][0:2]), pred2gps(choice_predgrids[i][2:])])
# print(choice_predgps)

' not used orz'

In [9]:
# # calculate the distance between prediction and ground truth
# start_dis = []; end_dis = []
# for i in range(len(choice_predgps)):
#     start_dis.append(distance(choice_predgps[i][0][1], choice_predgps[i][0][0], choice_testgps[i][0][1], choice_testgps[i][0][0]))
#     end_dis.append(distance(choice_predgps[i][1][1], choice_predgps[i][1][0], choice_testgps[i][1][1], choice_testgps[i][1][0]))
# print(start_dis)
# print(end_dis)
# print('average start distance: {} km'.format(np.mean(start_dis)))
# print('average end distance: {} km'.format(np.mean(end_dis)))

# with open("./results/{}_{}_{}_dists".format(emb_model, emb_dim, 512), "w") as f:
#     f.write("choose 40 points by random\n")
#     f.write("choice_list: {}\n".format(choice_list))
#     f.write('average start distance: {} km'.format(np.mean(start_dis)))
#     f.write('average end distance: {} km'.format(np.mean(end_dis)))
#     f.write('\n')
#     f.write('start distance: {}'.format(start_dis))
#     f.write('\n')
#     f.write('end distance: {}'.format(end_dis))

In [23]:
# if you want, plot the prediction and ground truth on the map
import folium
my_map = folium.Map(location=[40.953673,-8.735152], zoom_start=12)
choice = np.random.choice(len(test_gps), 1, replace=False)

print(test_gps[choice[0]])
print(test_predgps_[choice[0]])
print(test_gps_[choice[0]])

# plot the chosen trajectory and the prediction on the map and need to exchange the lon and lat
for i in choice:
    folium.CircleMarker(location=[test_gps_[i][0][1], test_gps_[i][0][0]], radius=5, color='red').add_to(my_map)
    folium.CircleMarker(location=[test_gps_[i][1][1], test_gps_[i][1][0]], radius=5, color='red').add_to(my_map)
    folium.CircleMarker(location=[test_predgps_[i][0][1], test_predgps_[i][0][0]], radius=5, color='blue').add_to(my_map)
    folium.CircleMarker(location=[test_predgps_[i][1][1], test_predgps_[i][1][0]], radius=5, color='blue').add_to(my_map)

    folium.PolyLine(locations=[[test_gps_[i][0][1], test_gps_[i][0][0]], [test_gps_[i][1][1], test_gps_[i][1][0]]], color='blue',
                    weight=15, opacity=0.6).add_to(my_map)
my_map 

[array([-8.61746113, 41.1454299 ]), array([-8.61714754, 41.14485466]), array([-8.61689005, 41.1448387 ]), array([-8.61602615, 41.1452544 ]), array([-8.6155424, 41.1454861]), array([-8.61483706, 41.14574917]), array([-8.61387475, 41.14599843]), array([-8.6135854, 41.1459177]), array([-8.61289667, 41.1459318 ]), array([-8.61219756, 41.14572197]), array([-8.61185873, 41.14508723]), array([-8.61122182, 41.14490574]), array([-8.61091747, 41.14514363]), array([-8.6108766 , 41.14552235]), array([-8.61077517, 41.1459019 ]), array([-8.6106166 , 41.14604373]), array([-8.61044055, 41.14616795]), array([-8.60936059, 41.14678558]), array([-8.60975784, 41.14761624]), array([-8.61111343, 41.14786573]), array([-8.61130633, 41.14711918]), array([-8.61100303, 41.14752277]), array([-8.6107309 , 41.14849798]), array([-8.61029584, 41.14994261]), array([-8.61010887, 41.15075823]), array([-8.60962241, 41.15142717]), array([-8.60988427, 41.15193947]), array([-8.6100328, 41.152706 ]), array([-8.6099982 , 41.15